<a href="https://colab.research.google.com/github/Kobzarko/python_for_DA_tasks/blob/main/hw_6_4_%D0%90%D0%BD%D0%B0%D0%BB%D1%96%D0%B7_%D0%90%D0%92_%D1%82%D0%B5%D1%81%D1%82%D1%83.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Аналіз A/B тесту

Маємо проаналізувати дані A/B тесту в популярній [грі Cookie Cats](https://www.facebook.com/cookiecatsgame). Це класична гра-головоломка в стилі «з’єднай три», де гравець повинен з’єднати плитки одного кольору, щоб очистити дошку та виграти рівень. На дошці також зображені співаючі котики :)

Під час проходження гри гравці стикаються з воротами, які змушують їх чекати деякий час, перш ніж вони зможуть прогресувати або зробити покупку в додатку. У цьому блоці завдань ми проаналізуємо результати A/B тесту, коли перші ворота в Cookie Cats було переміщено з рівня 30 на рівень 40. Зокрема, ми проаналізуємо вплив на утримання (retention) гравців. Тобто хочемо зрозуміти чи переміщення воріт на 10 рівнів пізніше якимось чином вплинуло на те, що користувачі перестають грати в гру раніше чи пізніше з точки зору кількості їх днів з моменту встановлення гри.

Будемо працювати з даними з файлу `cookie_cats.csv`. Змінні в даних наступні:

- userid - унікальний номер, який ідентифікує кожного гравця.
- version - чи потрапив гравець в контрольну групу (gate_30 - ворота на 30 рівні) чи тестову групу (gate_40 - ворота на 40 рівні).
- sum_gamerounds - кількість ігрових раундів, зіграних гравцем протягом першого тижня після встановлення
- retention_1 - чи через 1 день після встановлення гравець повернувся і почав грати?
- retention_7 - чи через 7 днів після встановлення гравець повернувся і почав грати?

Коли гравець встановлював гру, його випадковим чином призначали до групи gate_30 або gate_40.

1. Зчитайте дані АВ тесту у змінну `df` та виведіть середнє значення показника показник `retention_7` (утримання на 7 день) по версіям гри. Сформулюйте гіпотезу: яка версія дає краще утримання через 7 днів після встановлення гри?

In [36]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import statsmodels.stats.api as sms
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from math import ceil
from statsmodels.stats.proportion import proportions_ztest

%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

data_path ='/content/drive/MyDrive/Colab Notebooks/Data/cookie_cats.csv'

df = pd.read_csv(data_path)

df.head()

Mounted at /content/drive


,userid,version,sum_gamerounds,retention_1,retention_7
0,116,gate_30,3,False,False
1,337,gate_30,38,True,False
2,377,gate_40,165,True,False
3,483,gate_40,1,False,False
4,488,gate_40,179,True,True


Перш ніж ми продовжимо і проаналізуємо дані для отримання нашої підмножини, давайте упевнимося, що немає користувачів, які були обрані кілька разів.

In [ ]:
session_counts = df['userid'].value_counts(ascending=False)
multi_users = session_counts[session_counts > 1].count()

print(f'Є {multi_users} користувачів, які зустрічаються кілька разів у наборі даних.')

Є 0 користувачів, які зустрічаються кілька разів у наборі даних.


In [28]:
# retention_7_group_30 = df[df['version']=='gate_30']
df_retention_7 = df.groupby('version')['retention_7'].mean()
df_retention_7


,retention_7
version,
gate_30,0.190201
gate_40,0.182000


In [ ]:
print(f"Середнє значення для групи group_30: {df_retention_7['gate_30'].round(4)}")
print(f"Середнє значення для групи group_40: {df_retention_7['gate_40'].round(4)}")

Середнє значення для групи group_30: 0.1902
Середнє значення для групи group_40: 0.182


#### Висновок по гепотизі 1
На основі цих середніх значень можна припустити, що версія gate_30 (з воротами на 30 рівні) дає краще утримання через 7 днів після встановлення гри порівняно з версією gate_40 (з воротами на 40 рівні)

2. Перевірте з допомогою z-тесту аналогічно до прикладу в лекції, чи дає якась з версій гри кращий показник `retention_7` на рівні значущості 0.05. Обчисліть також довірчі інтервали для двох вибірок. Виведіть результат у форматі:
```
z statistic: ...
p-value: ...
Довірчий інтервал 95% для групи control: [..., ...]
Довірчий інтервал 95% для групи treatment: [..., ...]
```
де замість `...` - обчислені значення. В якості висновка дайте відповідь на два питання:  
    1. чи є статистична значущою різниця між поведінкою користувачів у різних версіях гри?   
    2. чи перетинаються довірчі інтервали утримання користувачів з різних версій гри? Про що це каже?  
    
Зверніть увагу, в такому і схожому завданнях ми використовуєм `proportion` Z-тест. Це тому що в нас залежна змінна має бінарне значення (повернеться аби ні користувач, чи клікне або ні користувач в інших ситуаціях - всього два можливих значення в змінної: 0/1, True/False ). Якщо б ми вимірювали скажімо чи є стат. значущою різниця між вагою чоловіків і жінок в певній вибірці, ми б використовувавли функцію `statsmodels.stats.ztest`, бо залежна змінна `вага` є неперервною (тип float, замість типу int чи bool і тільки двох можливих значень).

In [37]:
# Поділ даних на контрольну (gate_30) і тестову (gate_40) групи
control = df[df['version'] == 'gate_30']
treatment = df[df['version'] == 'gate_40']

# Кількість успіхів у обох групах (повернення на 7-й день)
control_successes = control['retention_7'].sum()
treatment_successes = treatment['retention_7'].sum()

# Кількість гравців у кожній групі
control_size = control.shape[0]
treatment_size = treatment.shape[0]

# Виконуємо Z-тест на пропорції
z_stat, p_value = proportions_ztest([control_successes, treatment_successes], [control_size, treatment_size])

# Виведення результатів
print(f'Z-статистика: {z_stat}')
print(f'P-значення: {p_value}')

# Довірчі інтервали для кожної групи
control_proportion = control_successes / control_size
treatment_proportion = treatment_successes / treatment_size

control_se = np.sqrt(control_proportion * (1 - control_proportion) / control_size)
treatment_se = np.sqrt(treatment_proportion * (1 - treatment_proportion) / treatment_size)

confidence_interval_control = [control_proportion - 1.96 * control_se, control_proportion + 1.96 * control_se]
confidence_interval_treatment = [treatment_proportion - 1.96 * treatment_se, treatment_proportion + 1.96 * treatment_se]

# Виведення довірчих інтервалів
print(f'Довірчий інтервал 95% для групи gate_30: {confidence_interval_control}')
print(f'Довірчий інтервал 95% для групи gate_40: {confidence_interval_treatment}')


Z-статистика: 3.164358912748191
P-значення: 0.001554249975614329
Довірчий інтервал 95% для групи gate_30: [0.18656304966751847, 0.1938396348962399]
Довірчий інтервал 95% для групи gate_40: [0.17845423557808565, 0.1855458523552609]


#### Висновки:
Статистична значущість: Так, оскільки p-value (0.00155) менше рівня значущості 0.05, ми можемо відхилити нульову гіпотезу. Це означає, що існує статистично значуща різниця між утриманням користувачів у двох версіях гри.

Перетин довірчих інтервалів: Довірчі інтервали для двох груп не перетинаються. Це свідчить про те, що різниця між версіями є не лише статистично значущою, але й суттєвою.

3. Є ще один тип тестів, який використовується для бінарної метрики як от "зробить юзер дію, чи ні" - тест [**Хі-квадрат**](https://www.bmj.com/about-bmj/resources-readers/publications/statistics-square-one/8-chi-squared-tests) (ще одне [пояснення](https://www.scribbr.com/statistics/chi-square-tests/) тесту з прикладами). В нього інші гіпотези Н0 і Н1 на відміну від z- та t-тестів. А також цей тест можна використовувати, якщо в нас більше за 2 досліджувані групи, тобто в нас не А/В тест, а А/B/C/D, наприклад.  

В **z- та t-тестах** (які відрізняються тим, що ми в першому не знаємо дисперсію генеральної сукупності, але якщо в нас великий набір даних, то ці два тести дають дуже схожі результати) **ми перевіряємо, чи є різниця у середніх показниках по групам користувачів**.  

А в **тесті Хі-квадрат ми перевіряємо чи є звʼязок між групою користувача і тим, чи він зробить цікаву нам дію**. Це ніби дослідження одного і того самого, але дещо різними способами. Для перевірки, можна виконувати кілька тестів (особливо, якщо один дає якийсь непереконливий результат типу р-значення 0.07 - наче і fail to regect H0 на рівні стат значущості 5%, але цікаво, що скажуть інші тести), тож, зробимо і ми тест хі-квадрат та порівняємо його результат з z-тестом.

Про різницю між тестами можна почитати ще [тут](https://stats.stackexchange.com/a/178860) - це просто пояснення користувача стековерфлоу, але там розумні люди сидять.

Для проведення хі-квадрат тесту скористаємось функцією з `scipy.stats` `chi2_contingency` для обчислення статистики хі-квадрат і р-значення для перевірки конкретної гіпотези. У цю функцію вам треба передати таблицю 2х2: кількість випадків для кожної версії гри і значення `retention_7`.

**Задача**: виконайте тест хі-квадрат на рівні значущості 5% аби визначити, чи є залежність між версією гри та тим, чи зайде гравець на 7ий день після встановлення гри.
Тут гіпотези наступні
- Н0: значення retention_7 не залежить від версії гри
- Н1: є залежність між версією гри і значенням retention_7

Виведіть p-значення та зробіть висновок.


In [39]:
# Створення таблиці 2x2
contingency_table = pd.crosstab(df['version'], df['retention_7'])
contingency_table

retention_7,False,True
version,,
gate_30,36198,8502
gate_40,37210,8279


In [40]:
# Виконання Хі-квадрат тесту
chi2, p, dof, expected = stats.chi2_contingency(contingency_table)

# Виведення результатів
print(f"Chi-square statistic: {chi2}")
print(f"p-value: {p}")

Chi-square statistic: 9.959086799559167
p-value: 0.0016005742679058301


In [41]:
# Перевірка рівня значущості
alpha = 0.05
if p < alpha:
    print("Відхиляємо H0: Є статистично значуща залежність між версією гри та значенням retention_7.")
else:
    print("Не можемо відхилити H0: Немає статистично значущої залежності між версією гри та значенням retention_7.")

Відхиляємо H0: Є статистично значуща залежність між версією гри та значенням retention_7.


#### Висновок:
Якщо p-значення менше 0.05, це означає, що існує залежність між версією гри та тим, чи повертається гравець на 7-й день. Якщо p-значення більше або дорівнює 0.05, то нульова гіпотеза не відхиляється.